In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from pycoalescence import CoalescenceTree

In [2]:
# best_sims_dir = "../../Results/Copied_files/"
best_sims_dir = "/run/media/sam/Media/Paleo/Results/PaleoMain/Copied_files"
data_dir = "../MainSimulationR/input/"
config_dir = "../../Data/configs/"
result_dir = "../MainSimulationR/results/"
seed_params = pd.read_csv(os.path.join(data_dir, "best_sim_parameters_seeds.csv"))
seed_params.head()

,Unnamed: 0,speciation_rate,scenario,sigma,deme,percent_cover,interval,tetrapod_group,seed,job_type
0,1,1.000000e-08,fragmented,10.162626,205,20,Artinskian,Amphibian,18,3
1,2,1.000000e-08,fragmented,16.119937,121,20,Asselian,Amniote,2,4
2,3,1.000000e-07,fragmented,16.824312,26,20,Asselian,Amniote,7,4
3,4,1.000000e-07,fragmented,16.824312,26,20,Asselian,Amniote,7,3
4,5,1.000000e-08,fragmented,16.119937,121,20,Asselian,Amphibian,2,5


In [7]:
output = []
equal_output = []
for i, row in tqdm_notebook(seed_params.iterrows(), desc="Files"):
    if row.percent_cover == 100:
        input_dir = os.path.join(best_sims_dir, "pristine")
    else:
        input_dir = os.path.join(best_sims_dir, str(row.percent_cover))
    input_path = os.path.join(input_dir, "data_{}_{}.db".format(row.job_type, row.seed))
    c = CoalescenceTree(input_path)
    c.wipe_data()
    sr = row.speciation_rate
    c.set_speciation_parameters(
        speciation_rates=sr,
        record_fragments=os.path.join(
            config_dir, "fragments_{}_{}.csv".format(row.interval.lower(), row.tetrapod_group.lower())
        ),
    )
    c.apply()
    c.calculate_fragment_richness()
    c.calculate_beta_diversity()
    species_richness = c.get_species_richness(1)
    beta = c.get_beta_diversity(1)
    alpha = c.get_alpha_diversity(1)
    total_ind = c.get_number_individuals(community_reference=1)
    interval = row.interval
    tet_group = row.tetrapod_group
    sigma = row.sigma
    deme = row.deme
    output.append(
        {
            "interval": interval,
            "tetrapod_group": tet_group,
            "sigma": sigma,
            "speciation_rate": sr,
            "deme": deme,
            "richness": species_richness,
            "beta_diversity": beta,
            "alpha_diversity": alpha,
            "percent_cover": row.percent_cover,
            "simulated_individuals": total_ind,
        }
    )
    if total_ind < 100:
        continue
    fragment_abundances = c.get_fragment_abundances_pd()
    fragment_abundances = fragment_abundances.reindex(
        fragment_abundances.index.repeat(fragment_abundances.no_individuals)
    )
    landscape_richness = []
    landscape_alphas = []
    for i in range(10):
        sampled_abundance = fragment_abundances.sample(n=100)
        mean_alpha = (
            sampled_abundance.groupby(by="fragment", as_index=False)
            .agg({"species_id": pd.Series.nunique})["species_id"]
            .mean()
        )
        species_richness = sampled_abundance.species_id.nunique()
        landscape_richness.append(species_richness)
        landscape_alphas.append(mean_alpha)
    mean_landscape_richness = np.mean(landscape_richness)
    mean_alpha = np.mean(landscape_alphas)
    mean_beta = mean_landscape_richness/mean_alpha
    equal_output.append(
        {
            "interval": interval,
            "tetrapod_group": tet_group,
            "sigma": sigma,
            "speciation_rate": sr,
            "deme": deme,
            "richness": mean_landscape_richness,
            "beta_diversity": mean_beta,
            "alpha_diversity": mean_alpha,
            "percent_cover": row.percent_cover,
            "simulated_individuals": total_ind,
        }
    )

In [8]:
output_df = pd.DataFrame(output)
output_df.to_csv(os.path.join(result_dir, "upscaled_best_fitting.csv"))

In [9]:
equal_output_df = pd.DataFrame(equal_output)
equal_output_df.to_csv(os.path.join(result_dir, "upscaled_sampled_best_fitting.csv"))

In [10]:
equal_output_df

,alpha_diversity,beta_diversity,deme,interval,percent_cover,richness,sigma,simulated_individuals,speciation_rate,tetrapod_group
0,2.026391,31.978036,205,Artinskian,20,64.8,10.162626,2486,1.000000e-08,Amphibian
1,6.946212,6.248010,121,Asselian,20,43.4,16.119937,370,1.000000e-08,Amniote
2,7.433333,8.663677,26,Asselian,20,64.4,16.824312,151,1.000000e-07,Amniote
3,7.191667,7.800695,26,Asselian,20,56.1,16.824312,151,1.000000e-07,Amniote
4,1.847974,25.541488,121,Asselian,20,47.2,16.119937,2110,1.000000e-08,Amphibian
5,1.871194,32.759827,205,Kungurian,20,61.3,10.162626,3181,1.000000e-08,Amphibian
6,7.516667,6.891353,134,Asselian,20,51.8,19.340737,358,1.000000e-08,Amniote
7,1.829066,34.225119,26,Kungurian,20,62.6,16.824312,478,1.000000e-07,Amphibian
8,1.767434,34.456735,177,Kungurian,20,60.9,11.861039,3043,1.000000e-08,Amniote
9,7.033333,7.393365,177,Asselian,20,52.0,11.861039,358,1.000000e-08,Amniote
